In [1]:
# 1. Основные библиотеки
import numpy as np
import pandas as pd
import pickle   # сохранение модели
import gc # сборщик мусора
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='svg'
import logging
import os
import subprocess
import time
import io, sys, traceback
#import argparse
import glob
import random
import signal
import wave
import soundfile as sf
from scipy.fftpack import dct
from python_speech_features import mfcc
import librosa

# импортирование библиотек моделей
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
#import xgboost as xgb

# импортирование библиотек для оптимизации моделей и оптимизации гиперпараметров
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, RandomizedSearchCV, learning_curve
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import missingno as msno

# импортирование метрик и репортинга
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

# убираем лишние замечания интерпретатора
import warnings
warnings.simplefilter('ignore')

#from python_speech_features import mfcc
pd.options.display.max_columns = 50

### 1. Downloading all sound samples in numpy massive, adding labels in the end

In [2]:
# name of folders(name of lables) and amount of samples (wav file) in each folder
#labels = {'go':3880,'backward':1664,'forward':1557,'left':3801, 'right':3778, 'stop':3872,'silience':2311}
labels = {'0':3880,'1':1664,'2':1557,'3':3801, '4':3778, '5':3872,'6':2311}

In [3]:
# to make datasets for each label
for label_key in labels.keys():

    df = pd.DataFrame(pd.Series(np.zeros(16000)), dtype=np.float16).T
    os.chdir(label_key)
    for i in range (labels.get(label_key)):
        sig, samplerate = sf.read(str(i))
        df.loc[i, :] = pd.Series(sig)
    sig, samplerate = sf.read('0')
    samplerate

    label = [int(label_key) for i in range(labels.get(label_key))]
    df['16000'] = np.array(label)
    X = df.drop('16000', axis=1)
    X[X.isna()] = 0
    y = df[['16000']]
    os.chdir('..')
    X.to_pickle(label_key+'.pkl')
    y.to_pickle('y_'+label_key+'.pkl')

In [181]:
df = pd.read_pickle('dataset.pkl')
df.shape

(20863, 16001)

In [182]:
y = pd.read_pickle('y_digits.pkl')
y.shape

(20863, 1)

In [183]:
X = df.drop('16000', axis=1)
X.shape

(20863, 16000)

### 2. Conversation of signal datasets into MFCC datasets

In [188]:
def mfcc(X):
    
    emphasized_signal = X.to_numpy().flatten()
    
    #HF increase filter realization, pre_emphasis = 0.97:
    #pre_emphasis = 0.97
    #emphasized_signal_ = np.append(X[0], X[1:] - pre_emphasis * X[:-1])

    #sclicing of speech by: frame_size = 0.04s and frame_stride = 0.04s
    frame_size = 0.04
    frame_stride = 0.04
    sample_rate = 16000
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step)) + 1  # Make sure that we have at least 1 frame, should 
                                                             #be 26 frames for whole signal (1 for tflite file input tensor (1,260)
    pad_signal_length = num_frames * frame_step + frame_length  # should be 16640
    z = np.zeros((pad_signal_length - signal_length))  # to fill np array with 0 size [640]
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any 
                                                 # samples from the original signal
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    
    # to apply Hemming window function:
    frames *= np.hamming(frame_length)
    # frames *= 0.54 - 0.46 * np.cos((2 * np.pi * n) / (frame_length - 1))  # Explicit Implementation **

    # spectrum and power spectrum transformation for each frame:
    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

    # filtering using triangle filters of Mel sclale nfilt = 21 (to get filter banks 20):
    nfilt = 41
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = np.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * np.log10(filter_banks)  # dB

    # to get mfcc for cepstrum, totally should rest = 10:
    num_ceps = 11
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : num_ceps] # Keep 2-11
    
    # may apply sinusoidal liftering to the MFCCs to de-emphasize higher MFCCs which to improve speech recognition in noisy signals:
    #cep_lifter = 22
    #(nframes, ncoeff) = mfcc.shape
    #n = np.arange(ncoeff)
    #lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
    #mfcc *= lift

    # The mean-normalized MFCCs
    #mfcc -= (np.mean(mfcc, axis=0) + 1e-8)
        
    X = mfcc.flatten()
    
    return X

In [185]:
MFCC = pd.DataFrame(pd.Series(np.zeros(250)), dtype=np.float16).T
    
# to convert signals dataset to MFCC dataset
for i in range (20863):
    MFCC.loc[i, :] = mfcc(X.iloc[i, :])
        
MFCC[MFCC.isna()] = 0
MFCC.to_pickle('MFCC.pkl')

scaler = MinMaxScaler()
MFCC_norm = MFCC.copy()
MFCC_norm = scaler.fit_transform(MFCC_norm)
MFCC_norm = pd.DataFrame(MFCC_norm)
MFCC_norm.to_pickle('MFCC_norm.pkl')

## EDA

In [186]:
X = pd.read_pickle('MFCC.pkl')
continuous_feature = X.select_dtypes(exclude=['object'])
continuous_feature.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            240, 241, 242, 243, 244, 245, 246, 247, 248, 249],
           dtype='int64', length=250)

In [187]:
# гистограммы распределения признаков:

fig, axes = plt.subplots(125, 2, figsize=(10, 300))

for num, feature in enumerate(continuous_feature):
    data = X[feature].copy()
    axes[num//2, num%2].hist(data, bins=50, color="blue", ec='white')
    axes[num//2, num%2].set_xlabel(feature, fontsize=5)

#### Logistic Regression

In [ ]:
from scipy.optimize import fmin_tnc
class LogisticRegressionUsingGD:

    @staticmethod
    def sigmoid(x):
        # Activation function used to map any real value between 0 and 1
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def net_input(theta, x):
        # Computes the weighted sum of inputs Similar to Linear Regression

        return np.dot(x, theta)

    def probability(self, theta, x):
        # Calculates the probability that an instance belongs to a particular class

        return self.sigmoid(self.net_input(theta, x))

    def cost_function(self, theta, x, y):
        # Computes the cost function for all the training samples
        m = x.shape[0]
        total_cost = -(1 / m) * np.sum(
            y * np.log(self.probability(theta, x)) + (1 - y) * np.log(
                1 - self.probability(theta, x)))
        return total_cost

    def gradient(self, theta, x, y):
        # Computes the gradient of the cost function at the point theta
        m = x.shape[0]
        return (1 / m) * np.dot(x.T, self.sigmoid(self.net_input(theta, x)) - y)

    def fit(self, x, y, theta):
        """trains the model from the training data
        Uses the fmin_tnc function that is used to find the minimum for any function
        It takes arguments as
            1) func : function to minimize
            2) x0 : initial values for the parameters
            3) fprime: gradient for the function defined by 'func'
            4) args: arguments passed to the function
        Parameters
        ----------
        x: array-like, shape = [n_samples, n_features]
            Training samples
        y: array-like, shape = [n_samples, n_target_values]
            Target classes
        theta: initial weights
        Returns
        -------
        self: An instance of self
        """

        opt_weights = fmin_tnc(func=self.cost_function, x0=theta, fprime=self.gradient,
                               args=(x, y.flatten()))
        self.w_ = opt_weights[0]
        return self

    def predict(self, x):
        """ Predicts the class labels
        Parameters
        ----------
        x: array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        predicted class labels
        """
        theta = self.w_[:, np.newaxis]
        return self.probability(theta, x)

    def accuracy(self, x, actual_classes, probab_threshold=0.5):
        """Computes the accuracy of the classifier
        Parameters
        ----------
        x: array-like, shape = [n_samples, n_features]
            Training samples
        actual_classes : class labels from the training data set
        probab_threshold: threshold/cutoff to categorize the samples into different classes
        Returns
        -------
        accuracy: accuracy of the model
        """
        predicted_classes = (self.predict(x) >= probab_threshold).astype(int)
        predicted_classes = predicted_classes.flatten()
        accuracy = np.mean(predicted_classes == actual_classes)
        return accuracy * 100

### Разбиение обучающего датасета на "обучающий" - X_train и тестовый - X_test

In [ ]:
for i in range(7):
    
    # to make one vs all other y_targets vectors for all combinations
    y = pd.read_pickle('y_digits.pkl')
    y.loc[y['16000'] != i, '16000'] = 1
    y.loc[y['16000'] == i, '16000'] = 0
    y.to_pickle(i+'.pkl')
    
    # to split dataset into train and test for all y_target vecors (one vs all others)
    X_train, X_test, y_train, y_test = train_test_split(MFCC, y, shuffle=True, test_size=0.2, random_state=21)
    
    # preparing the data for building the model
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    y_train = y_train.to_numpy()[:,np.newaxis]
    theta = np.zeros((X_train.shape[1], 1))
    
    # Logistic Regression from scratch using Gradient Descent
    model = LogisticRegressionUsingGD()
    model.fit(X_train, y_train, theta)
    accuracy = model.accuracy(X_train, y_train.flatten())
    parameters = model.w_
    print("The accuracy of the model is {}".format(accuracy))
    print("The model parameters using Gradient descent")
    print("\n")
    
    # preparing the data for prediction the model
    X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
    y_test = y_test.to_numpy()[:,np.newaxis]
    theta = np.zeros((X_test.shape[1], 1))

    # calculation accuracy of y_test
    accuracy = model.accuracy(X_test, y_test.flatten())
    print("The accuracy of the model is {}".format(accuracy))

In [242]:
y = pd.read_pickle('y_digits.pkl')

In [244]:
X_train, X_test, y_train, y_test = train_test_split(MFCC, y, shuffle=True, test_size=0.2, random_state=21)

#### Удалим ненужные файлы
del df
gc.collect()

In [245]:
# preparing the data for building the model

X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
y_train = y_train.to_numpy()[:,np.newaxis]
theta = np.zeros((X_train.shape[1], 1))

In [246]:
# Logistic Regression from scratch using Gradient Descent
    
model = LogisticRegressionUsingGD()
model.fit(X_train, y_train, theta)
accuracy = model.accuracy(X_train, y_train.flatten())
parameters = model.w_
print("The accuracy of the model is {}".format(accuracy))
print("The model parameters using Gradient descent")
print("\n")
print(parameters)

The accuracy of the model is 76.73457159976034
The model parameters using Gradient descent


[ 3.27170605e-04  7.03403982e-03  1.72550735e-03  3.02290274e-03
 -1.53989767e-03  1.26908858e-03 -2.97479899e-03 -4.37694280e-04
 -2.54538186e-03 -4.02861002e-04 -2.66216504e-03  7.94317546e-03
  1.68260932e-03  3.10989909e-03 -1.99202602e-03  1.24057039e-03
 -3.04524562e-03 -5.51238598e-04 -2.56891115e-03 -2.76210686e-04
 -2.68169634e-03  9.54869072e-03  1.63844239e-03  2.75183879e-03
 -2.82354741e-03  1.04274201e-03 -3.13425945e-03 -8.14963499e-04
 -2.47068713e-03 -1.96393661e-04 -2.58554547e-03  1.14044804e-02
  1.54668709e-03  2.77189360e-03 -3.54215283e-03  4.30890897e-04
 -3.31045184e-03 -1.37676195e-03 -2.30123716e-03  1.76959256e-05
 -2.44508817e-03  1.34304489e-02  8.34803520e-04  2.92346158e-03
 -4.41168320e-03  5.36875710e-05 -3.51587562e-03 -1.75079491e-03
 -2.54258999e-03  1.09168007e-05 -2.72922856e-03  1.48218961e-02
  5.07003580e-05  3.27598445e-03 -4.92198759e-03 -4.49218166e-

In [247]:
parameters.shape

(251,)

In [248]:
# preparing the data for prediction the model

X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
y_test = y_test.to_numpy()[:,np.newaxis]
theta = np.zeros((X_test.shape[1], 1))

# calculation accuracy of y_test

accuracy = model.accuracy(X_test, y_test.flatten())
parameters = model.w_
print("The accuracy of the model is {}".format(accuracy))

The accuracy of the model is 75.84471603163192


### Создание и обучение моделей

In [ ]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [ ]:
model_lr = LogisticRegression(multi_class='auto',C=5,random_state=21)
model_lr.fit(X_train, y_train)

y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test.to_numpy().flatten(), y_test_pred)

In [ ]:
model_lr.fit(MFCC_norm, y)

In [ ]:
with open('model_lr_norm','wb') as file:
    pickle.dump(model_lr,file)

### Oценка выбранной модели - Logistic Regression

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, r2_score

In [ ]:
y_test_preds = pd.DataFrame(); y_test_preds = model_lr.predict(X_test)

In [ ]:
y_test_preds

In [ ]:
accuracy_score(y_test,y_test_preds)

In [ ]:
y_train_preds = pd.DataFrame(); y_train_preds = model_lr.predict(X_train)

In [ ]:
accuracy_score(y_train,y_train_preds)

In [ ]:
f1_score(y_test,y_test_preds, average='micro')